In [1]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))
    
%run -i 'implicit.py'

# Crandall-Rabinowitz

We now have a readily callable implicit function theorem, lets put it to work and automate the method of Crandall-Rabinowitz

Crandall-Rabinowitz pertains to the following case:

$$F : \mathbb{R} \times X \rightarrow Y $$

$$ X = \mathbb{R}^n $$

Where we have that $\partial_{x}F$ has Fredholm index 0 and a one-dimensional kernel.

From this we can recover that:

$$dim X = dim Y$$

and:

$$ \text{dimRange} \;\partial_{x}F = n-1 $$

### Changing basis

To begin to untangle the parts we can deal with the implict function theorem, consider the following bases for $X$ and $Y$:

$(u_{1}, \dots, u_{n-1}, \tilde{u}_{n})$ and $(v_{1}, \dots, v_{n-1}, \tilde{v}_{n})$

such that:

$$ \text{Ker} \;\partial_{x}F = \left< \tilde{u}_{n}\right>$$
$$ \text{Range} \;\partial_{x}F = \left< v_{1}, \cdots, v_{n-1} \right>$$

Let $A$ and $B \in \mathcal{L}(\mathbb{R}^n, \mathbb{R}^n)$ be the respective change of basis matrices

See that $$\tilde{F} = B^{-1} \circ F \circ A$$

Now has one input variable acting in the kernel and one output not in the range, (abusing notation here, $A$,$B$ leave the $\mathbb{R}$ input alone)

### Worked example

To build up the method lets use a previous example:

In [2]:
var('x y z')
var('l', latex_name=r'\lambda')
f1(x,y,z,l) = x*sin(sqrt(x^2 + y^2 +  (pi^2)/9)) - y*cos(sqrt(x^2 + y^2 +  (pi^2)/9)) + exp(z) - 1 # zeros when z = 0
f2(x,y,z,l) = x*cos(l) + y*sin(l) + z 
f3(x,y,z,l) = z^2 + 4*z + z*l + x^2 + y^3 # zeros out at when X = 0   [X = <x,y,z>]
lprint(f1)
lprint(f2)
lprint(f3)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -y \cos\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + x \sin\left(\frac{1}{3} \, \sqrt{\pi^{2} + 9 \, x^{2} + 9 \, y^{2}}\right) + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ x \cos\left({\lambda}\right) + y \sin\left({\lambda}\right) + z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ y^{3} + x^{2} + {\lambda} z + z^{2} + 4 \, z $

We aim to describe the solution set when $\lambda = -\frac{\pi}{6} $, and determine that it is indeed a bifucation point

In [3]:
J = jacobian([f1,f2,f3],(x,y,z))(x=0,y=0,z=0,l=-pi/6)
lprint(J)

$ \left(\begin{array}{rrr}
\frac{1}{2} \, \sqrt{3} & -\frac{1}{2} & 1 \\
\frac{1}{2} \, \sqrt{3} & -\frac{1}{2} & 1 \\
0 & 0 & -\frac{1}{6} \, \pi + 4
\end{array}\right) $

In [4]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,\sqrt{3},\,0\right)\right] $

In [5]:
I = J.image().basis()
lprint(I)

$ \left[\left(1,\,-\frac{1}{3} \, \sqrt{3},\,0\right), \left(0,\,0,\,1\right)\right] $

#### Extending to bases

We can use this trick as per math stackexhange https://math.stackexchange.com/questions/465870/how-to-extend-a-basis

In [6]:
Kperp = matrix(K).right_kernel().basis()
lprint(Kperp)

$ \left[\left(1,\,-\frac{1}{3} \, \sqrt{3},\,0\right), \left(0,\,0,\,1\right)\right] $

In [7]:
Iperp = matrix(I).right_kernel().basis()
lprint(Iperp)

$ \left[\left(1,\,\sqrt{3},\,0\right)\right] $

#### Getting the A,B matrices

In [8]:
A = matrix(K+Kperp).inverse()
lprint(A) # inverse so is change of basis

$ \left(\begin{array}{rrr}
\frac{1}{4} & \frac{3}{4} & 0 \\
\frac{1}{4} \, \sqrt{3} & -\frac{1}{4} \, \sqrt{3} & 0 \\
0 & 0 & 1
\end{array}\right) $

In [9]:
B = matrix(I + Iperp).inverse()
lprint(B)

$ \left(\begin{array}{rrr}
\frac{3}{4} & 0 & \frac{1}{4} \\
-\frac{1}{4} \, \sqrt{3} & 0 & \frac{1}{4} \, \sqrt{3} \\
0 & 1 & 0
\end{array}\right) $

These are the required matrices for the formulation above

#### The new f

Now we can setup a new function $\tilde{f}$ that behaves as we need to apply the implicit function code - in all but the first slot that is

In [10]:
f1A(x,y,z,l) = f1(*list(A*vector([x,y,z])),l)
f2A(x,y,z,l) = f2(*list(A*vector([x,y,z])),l)
f3A(x,y,z,l) = f3(*list(A*vector([x,y,z])),l)
lprint(f1A)
lprint(f2A)
lprint(f2A)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{4} \, {\left(x + 3 \, y\right)} \cos\left({\lambda}\right) + \frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left({\lambda}\right) + z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{4} \, {\left(x + 3 \, y\right)} \cos\left({\lambda}\right) + \frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left({\lambda}\right) + z $

In [11]:
[ft1,ft2,ft3] = B.inverse()*vector([f1A,f2A,f3A])
lprint(ft1)
lprint(ft2)
lprint(ft3)

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) - \frac{1}{12} \, \sqrt{3} {\left({\left(x + 3 \, y\right)} \cos\left({\lambda}\right) + {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left({\lambda}\right) + 4 \, z\right)} + e^{z} - 1 $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ \frac{1}{64} \, {\left(\sqrt{3} x - \sqrt{3} y\right)}^{3} + \frac{1}{16} \, {\left(x + 3 \, y\right)}^{2} + {\lambda} z + z^{2} + 4 \, z $

$ \left( x, y, z, {\lambda} \right) \ {\mapsto} \ -\frac{1}{4} \, {\left(\sqrt{3} x - \sqrt{3} y\right)} \cos\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, {\left(x + 3 \, y\right)} \sin\left(\frac{1}{6} \, \sqrt{4 \, \pi^{2} + 9 \, x^{2} + 27 \, y^{2}}\right) + \frac{1}{4} \, \sqrt{3} {\left({\left(x + 3 \, y\right)} \cos\left({\lambda}\right) + {\left(\sqrt{3} x - \sqrt{3} y\right)} \sin\left({\lambda}\right) + 4 \, z\right)} + e^{z} - 1 $

In [12]:
Jt = jacobian([ft1,ft2,ft3],(x,y,z))(x=0,y=0,z=0,l=-pi/6)
lprint(Jt)

$ \left(\begin{array}{rrr}
0 & \frac{1}{2} \, \sqrt{3} - \frac{1}{2} & -\frac{1}{3} \, \sqrt{3} + 1 \\
0 & 0 & -\frac{1}{6} \, \pi + 4 \\
0 & \frac{1}{2} \, \sqrt{3} + \frac{3}{2} & \sqrt{3} + 1
\end{array}\right) $

In [13]:
Kt = Jt.right_kernel().basis()
lprint(Kt)

$ \left[\left(1,\,0,\,0\right)\right] $

In [14]:
It = Jt.image().basis()
lprint(It)

$ \left[\left(0,\,1,\,0\right), \left(0,\,0,\,1\right)\right] $

Success! - we see that now the kernel and image directions are teased out.

### Running the implicit function code

The implicit function method only works for the case when the $f_{Y}$ partial is invertible, in this case that means we omit $\lambda$ and the kernel direction, which for our $\tilde{f}$ means ingnoring $x$

Similarly we omit $\tilde{f}_{1}$ from consideration from the implicit function theorem, since this traverses the cokernel of the Jacobian

Translating into the language of the implicit function theorem code, somewhat confusingly we set:

$y_{1} = y$

$y_{2} = z$

$x_{1} = \lambda$

$x_{2} = x$


In [47]:
funcs = [ft2, ft3]
position = {'x' : 0, 'y' : 0, 'z' : 0, 'l' : -pi/6} # drawback that have to write this way
var_dict = {'y1' : y, 'y2' : z, 'x1' : l, 'x2' : x}
x_var_keys = ['x1','x2']
x_dim = 2
y_dim = 2

In [48]:
t_dict = TensorDict(funcs, position, var_dict, x_dim, y_dim)

In [53]:
out = get_hkx_polynomial(funcs, 2, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

[0, 0]
[0, 0]


In [54]:
lprint(out)

$ \left[-\frac{\sqrt{3} {\lambda} x}{\sqrt{3} + 3} - \frac{3 \, x^{2} {\left(\sqrt{3} + 1\right)}}{4 \, {\left(\pi - 24\right)} {\left(\sqrt{3} + 3\right)}}, \frac{3 \, x^{2}}{8 \, {\left(\pi - 24\right)}}\right] $

This is the parametrisation of the zero set of 